In [1]:
# --- 1. 安装和导入必要的库 ---
# 请确保已安装: pip install scanpy webdataset torch torchvision Pillow torch_geometric
import os
import scanpy as sc
import webdataset as wds
import torch
from torch.utils.data import DataLoader
import torch_geometric.data as pyg_data
from PIL import Image
import io
import random
from tqdm.auto import tqdm

# --- 2. 路径配置 (请根据您的环境修改) ---
BASE_DIR = "/cwStorage/nodecw_group/jijh"
OUTPUT_DIR = os.path.join(BASE_DIR, "spaglam_sota_data")
FINAL_ADATA_PATH = os.path.join(OUTPUT_DIR, "master_adata_with_graph_and_paths.h5ad")
SHARDS_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "webdataset_shards")

# --- 3. 加载主 AnnData 文件 ---
# 这个文件是我们的“中央索引”，在整个调试过程中都会使用
print(f"正在加载主 AnnData 文件: {FINAL_ADATA_PATH}")
if not os.path.exists(FINAL_ADATA_PATH):
    raise FileNotFoundError("错误: 找不到主 AnnData 文件。请先运行预处理脚本。")
adata = sc.read_h5ad(FINAL_ADATA_PATH)
print("✅ 主 AnnData 文件加载成功！")

# 打印一些信息以供确认
print(f"维度 (spots, genes): {adata.shape}")
print(f"包含的列: {adata.obs.columns.tolist()}")
print(f"包含的图: {'spatial_connectivities' in adata.obsp}")

正在加载主 AnnData 文件: /cwStorage/nodecw_group/jijh/spaglam_sota_data/master_adata_with_graph_and_paths.h5ad
✅ 主 AnnData 文件加载成功！
维度 (spots, genes): (997054, 30148)
包含的列: ['in_tissue', 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_col', 'array_row', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'sample_id', 'pxl_row_in_fullres_old', 'pxl_col_in_fullres_old', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'image_path', 'sentence_path']
包含的图: True


# 单样本加载函数

In [2]:
def get_spaglam_sample(spot_id: str, adata: sc.AnnData):
    """
    一个用于调试的独立函数，加载单个中心点及其邻居的所有数据。
    
    参数:
        spot_id (str): 我们感兴趣的中心点的ID。
        adata (sc.AnnData): 已加载的主 AnnData 对象。
        
    返回:
        一个包含所有相关信息的字典，或在出错时返回 None。
    """
    print(f"\n--- 正在处理 Spot ID: {spot_id} ---")
    
    try:
        # --- 步骤 1: 从 AnnData 中获取元数据和邻居 ---
        center_idx = adata.obs_names.get_loc(spot_id)
        
        # 获取邻居索引
        neighbor_indices = adata.obsp['spatial_connectivities'][center_idx].indices
        neighbor_ids = adata.obs_names[neighbor_indices].tolist()
        
        all_spot_ids = [spot_id] + neighbor_ids
        print(f"找到 {len(neighbor_ids)} 个邻居: {neighbor_ids[:3]}...") # 打印前3个邻居

        # --- 步骤 2: 获取所有相关点的文件路径 ---
        paths_df = adata.obs.loc[all_spot_ids, ['image_path', 'sentence_path']]
        
        # --- 步骤 3: 从磁盘加载原始数据 ---
        images = []
        sentences = []
        for sid in all_spot_ids:
            img_path = paths_df.loc[sid, 'image_path']
            sent_path = paths_df.loc[sid, 'sentence_path']
            
            # 加载图像
            if os.path.exists(img_path):
                images.append(Image.open(img_path).convert("RGB"))
            else:
                print(f"警告: 图像文件未找到 {img_path}")
                images.append(Image.new('RGB', (224, 224), color = 'red')) # 用红色图像表示错误

            # 加载句子
            if os.path.exists(sent_path):
                with open(sent_path, 'r') as f:
                    sentences.append(f.read())
            else:
                print(f"警告: 句子文件未找到 {sent_path}")
                sentences.append("FILE_NOT_FOUND")

        # --- 步骤 4: 构建图结构 (简化版) ---
        # 实际Dataloader中会更高效
        num_nodes = len(all_spot_ids)
        # 假设是全连接图进行演示
        edge_index = torch.combinations(torch.arange(num_nodes), r=2).t().contiguous()
        
        print("✅ 成功加载了中心点及其邻居的数据。")
        
        return {
            "center_spot_id": spot_id,
            "images": images,
            "sentences": sentences,
            "graph_edge_index": edge_index
        }

    except Exception as e:
        print(f"❌ 处理 {spot_id} 时发生错误: {e}")
        return None

# --- 现在来测试一下这个函数 ---
random_spot_id = random.choice(adata.obs_names.tolist())
sample_data = get_spaglam_sample(random_spot_id, adata)

if sample_data:
    print(f"\n--- 验证结果 ---")
    print(f"中心点ID: {sample_data['center_spot_id']}")
    print(f"加载的图像数量: {len(sample_data['images'])}")
    print(f"第一张图像的尺寸: {sample_data['images'][0].size}")
    print(f"加载的句子数量: {len(sample_data['sentences'])}")
    print(f"第一条句子内容: '{sample_data['sentences'][0][:60]}...'")
    print(f"图的边索引形状: {sample_data['graph_edge_index'].shape}")


--- 正在处理 Spot ID: MISC59_GATCTTGGAGGGCATA-1 ---
找到 5 个邻居: ['MISC59_AGCGTGGTATTCTACT-1', 'MISC59_CTAAGGGAATGATTGG-1', 'MISC59_GCTACGACTTATTGGG-1']...
✅ 成功加载了中心点及其邻居的数据。

--- 验证结果 ---
中心点ID: MISC59_GATCTTGGAGGGCATA-1
加载的图像数量: 6
第一张图像的尺寸: (364, 364)
加载的句子数量: 6
第一条句子内容: 'S100A6-1 SLC25A5-1 ALDOA S100A14-1 ANPEP ATP5IF1 CDC42EP5 IT...'
图的边索引形状: torch.Size([2, 15])


# 【SOTA实现】构建生产级的 IterableDataset

In [4]:
# 假设您已经定义了图像预处理器和分词器
# from open_clip import get_tokenizer
# from open_clip.transform import image_transform_v2
# image_processor = image_transform_v2(...)
# tokenizer = get_tokenizer(...)

# 为了调试，我们先用伪函数代替
def dummy_image_processor(img):
    return torch.randn(3, 224, 224)

def dummy_tokenizer(texts):
    if isinstance(texts, str): texts = [texts]
    return torch.randint(0, 49408, (len(texts), 77))

# --- 定义 SOTA 数据集类 ---
class SotaSpatialWdsDataset(torch.utils.data.IterableDataset):
    def __init__(self, tar_urls, anndata_path, image_processor, tokenizer, config):
        super().__init__()
        self.tar_urls = tar_urls
        self.anndata_path = anndata_path
        self.image_processor = image_processor
        self.tokenizer = tokenizer
        self.config = config
        
        # AnnData将在每个工作进程中独立加载
        self.adata = None

    def _init_worker(self):
        """每个工作进程的初始化函数"""
        if self.adata is None:
            self.adata = sc.read_h5ad(self.anndata_path)

    def _get_neighbors(self, sample):
        """WebDataset流水线中的一个map操作，用于查找邻居"""
        self._init_worker() # 确保adata已加载
        
        center_key = sample['__key__']
        try:
            center_idx = self.adata.obs_names.get_loc(center_key)
            neighbor_indices = self.adata.obsp['spatial_connectivities'][center_idx].indices
            neighbor_keys = self.adata.obs_names[neighbor_indices].tolist()
            
            sample['all_keys'] = [center_key] + neighbor_keys
            # 这里可以进一步添加局部图结构
        except KeyError:
            # 如果key在AnnData中找不到，就跳过这个样本
            sample['all_keys'] = []
        return sample

    def __iter__(self):
        # 定义WebDataset数据处理流水线
        pipeline = wds.DataPipeline(
            wds.ResampledShards(self.tar_urls), # 1. 随机选择一个.tar文件
            wds.split_by_worker,               # 2. 将分片分配给不同的worker
            wds.tarfile_to_samples(),          # 3. 从tar中解包样本
            wds.select(lambda x: len(x['png']) > 0), # 过滤掉损坏的样本
            wds.map(self._get_neighbors),      # 4. 【核心】为每个中心点查找邻居key
            wds.select(lambda x: len(x['all_keys']) > 0), # 过滤掉没找到邻居的样本
            wds.map_dict(png=self.image_processor), # 5. 对中心点的图像进行预处理
            # 注意：此处简化了邻居数据的加载，实际项目中可能需要更复杂的
            # wds.associate 或 wds.map_group 来加载邻居数据。
            # 但对于调试和验证，这个流程是可行的。
        )
        return iter(pipeline)

# --- 实例化并测试数据集 ---
print("\n--- 实例化并测试生产级Dataset ---")
# 获取所有分片的URL列表
all_shards = [os.path.join(SHARDS_OUTPUT_PATH, f) for f in os.listdir(SHARDS_OUTPUT_PATH) if f.endswith('.tar')]

# 模拟的配置
N_NEIGHBORS = 6  # 假设每个中心点有6个邻居
mock_config = {"gnn_neighbors": N_NEIGHBORS, "batch_size": 4}

# 创建数据集实例
dataset = SotaSpatialWdsDataset(
    tar_urls=all_shards,
    anndata_path=FINAL_ADATA_PATH,
    image_processor=dummy_image_processor,
    tokenizer=dummy_tokenizer,
    config=type('MockConfig', (), mock_config)
)

# 使用DataLoader加载数据
# num_workers=0 意味着在主进程中运行，便于调试
dataloader = DataLoader(dataset, batch_size=mock_config['batch_size'], num_workers=0, collate_fn=lambda x: x)

# 取一个批次的数据进行检查
batch = next(iter(dataloader))

print(f"\n✅ 成功从Dataloader中获取了一个批次的数据！")
print(f"批次大小: {len(batch)}")

first_sample_in_batch = batch[0]
print("\n--- 检查批次中的第一个样本 ---")
print(f"样本包含的键: {first_sample_in_batch.keys()}")
print(f"中心点 key: {first_sample_in_batch['__key__']}")
print(f"邻居+中心点 keys: {first_sample_in_batch['all_keys']}")
print(f"处理后的图像张量形状: {first_sample_in_batch['png'].shape}")


--- 实例化并测试生产级Dataset ---

✅ 成功从Dataloader中获取了一个批次的数据！
批次大小: 4

--- 检查批次中的第一个样本 ---
样本包含的键: dict_keys(['__key__', '__url__', 'png', 'txt', 'all_keys'])
中心点 key: NCBI776_GCGAATCGAGAACACG-1
邻居+中心点 keys: ['NCBI776_GCGAATCGAGAACACG-1', 'NCBI776_AAGGCCGACCTACCTG-1', 'NCBI776_CTATGCCGTACAGCGT-1', 'NCBI776_CTTCGATAACATTGGT-1', 'NCBI776_TAGGATGCACCGTTCA-1', 'NCBI776_TGCATTGCTGTCGGCG-1']
处理后的图像张量形状: torch.Size([3, 224, 224])


In [5]:
# 取一个批次的数据进行检查
batch = next(iter(dataloader))

print(f"\n✅ 成功从Dataloader中获取了一个批次的数据！")
print(f"批次大小: {len(batch)}")

first_sample_in_batch = batch[1]
print("\n--- 检查批次中的第一个样本 ---")
print(f"样本包含的键: {first_sample_in_batch.keys()}")
print(f"中心点 key: {first_sample_in_batch['__key__']}")
print(f"邻居+中心点 keys: {first_sample_in_batch['all_keys']}")
print(f"处理后的图像张量形状: {first_sample_in_batch['png'].shape}")


✅ 成功从Dataloader中获取了一个批次的数据！
批次大小: 4

--- 检查批次中的第一个样本 ---
样本包含的键: dict_keys(['__key__', '__url__', 'png', 'txt', 'all_keys'])
中心点 key: INT4_TGCCGTGGATCGTCCT-1
邻居+中心点 keys: ['INT4_TGCCGTGGATCGTCCT-1', 'INT4_ACCCGGAAACTCCCAG-1', 'INT4_ATACCTAACCAAGAAA-1', 'INT4_CGATTAAATATCTCCT-1', 'INT4_GTGGACGCATTTGTCC-1', 'INT4_TGCTCGGCGAAACCCA-1']
处理后的图像张量形状: torch.Size([3, 224, 224])
